In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import binom_test

pd.set_option('display.max_columns', None)


# GT:AD:DP:GQ:JL:JP:PGT:PID:PL:PP:PS
# Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|
# BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Ami
# no_acids|Codons|Existing_variation|DISTANCE|STRAND|FLAGS|VARIANT_CLASS|SYMBOL_S
# OURCE|HGNC_ID|CANONICAL|MANE_SELECT|MANE_PLUS_CLINICAL|TSL|APPRIS|CCDS|ENSP|SWI
# SSPROT|TREMBL|UNIPARC|UNIPROT_ISOFORM|GENE_PHENO|SIFT|PolyPhen|DOMAINS|miRNA|HG
# VS_OFFSET|AF|AFR_AF|AMR_AF|EAS_AF|EUR_AF|SAS_AF|AA_AF|EA_AF|gnomAD_AF|gnomAD_AF
# R_AF|gnomAD_AMR_AF|gnomAD_ASJ_AF|gnomAD_EAS_AF|gnomAD_FIN_AF|gnomAD_NFE_AF|gnom
# AD_OTH_AF|gnomAD_SAS_AF|MAX_AF|MAX_AF_POPS|CLIN_SIG|SOMATIC|PHENO|PUBMED|MOTIF_
# NAME|MOTIF_POS|HIGH_INF_POS|MOTIF_SCORE_CHANGE|TRANSCRIPTION_FACTORS|CADD_phred
# |MPC_rankscore|MPC_score|MetaLR_pred|MetaLR_rankscore|MetaLR_score|MetaSVM_pred
# |MetaSVM_rankscore|Polyphen2_HDIV_score|REVEL_rankscore|REVEL_score|SIFT_pred|g
# nomAD_exomes_AC|gnomAD_exomes_AF|gnomAD_exomes_AN

In [2]:
annotated_vcf = '/projects/ps-gleesonlab8/User/arzoo/20230810_Jiny_Rerun_DNM_WGS_GMKF/1.Processing//output/merged_vcfs/annovar/joint.multisplit.VQSR.CGP.ann.vep.DNM.spliceAI.annotated.hg38_multianno.txt.reorder.vcf'

# Loading in DeNovos

In [3]:
families = {}
sample_lines = {}
de_novo_info = []
vep_header = []
spliceAI_header = []
greenvaran_header = [
    'greendb_id',
    'greendb_stdtype',
    'greendb_dbsource',
    'greendb_genes',
    'greendb_constraint',
    'greendb_level',
    'greendb_more_support'
]

with open('/projects/ps-gleesonlab8/User/arzoo/GMKF_full_ped/complete_peds/20230216_GMKF_NTD_WGS_trios_fixed.ped') as ped:
    for line in ped:
        info = line[:-1].split('\t')
        if info[-1] == '2':
            proband = info[1]
            father = info[2]
            mother = info[3]
            families[proband] = [father,mother,info[0]]


with open(annotated_vcf) as vcf:
    for line in vcf:
        if line[0:2] != '##' or line[0:14] == '##INFO=<ID=CSQ' or line.startswith('##INFO=<ID=SpliceAI'):
            if line[0:14] == '##INFO=<ID=CSQ':
                vep_header=line.split(': ')[1][:-3].split('|')
            elif line.startswith('##INFO=<ID=SpliceAI'):
                spliceAI_header=line.split(': ')[-1][:-3].split('|')[2:]
            if line[0] == '#':
                head = line[:-1].split('\t')
                for i in np.arange(9,len(head)):
                    sample_lines[head[i]] = i
            else:
                record = line[:-1].split('\t')
                locus = record[0] + ':' + record[1]
                ref = record[3]
                alt = record[4]
                PASS = record[6]
                variant_splice_AI = ['','','','','','','','']
                green_db = []
                for i in record[7].split(';'):
                    if i.split('=')[0] == "MQ":
                        variant_MQ = i.split('=')[1]
                    elif i.split('=')[0] == 'CSQ':
                        variant_vep = i.split('=')[1].split(',')[0].split('|')
                    elif i.split('=')[0] == 'SpliceAI':
                        variant_splice_AI = i.split('=')[1].split(',')[0].split('|')[2:]
                    elif i.split('=')[0].startswith('greendb_'):
                        green_db.append(i.split('=')[1])
                if green_db == []:
                    green_db = ['','','','','','','']
                for i in record[7].split(';'):
                    if "ConfDeNovo" in i:
                        probands = i.split('=')[1].split(',')
                        conf = i.split('=')[0]
                        for proband in probands:
                            proband_info = record[sample_lines[proband]]
                            family = families[proband][2]
                            proband_DP = proband_info.split(':')[2]
                            proband_GQ = proband_info.split(':')[3]
                            father_info = record[sample_lines[families[proband][0]]]
                            mother_info = record[sample_lines[families[proband][1]]]
                            father_DP = father_info.split(':')[2]
                            mother_DP = mother_info.split(':')[2]
                            de_novo_info.append([locus,ref,alt,PASS,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ] + variant_vep + variant_splice_AI + green_db + [record[-1]])

In [4]:
df = pd.DataFrame(de_novo_info,columns=['locus','ref','alt','filter','proband','family','conf','proband_info','father_info','mother_info','proband_GQ','proband_DP','father_DP','mother_DP','variant_MQ'] + vep_header + spliceAI_header + greenvaran_header + ['gnomad_genomes_312_AF'])

In [5]:
convert_dict = {
    'locus': str,
    'ref': str,
    'alt': str,
    'filter': str,
    'proband': str,
    'family': str,
    'conf': str,
    'proband_info': str,
    'father_info': str,
    'mother_info': str,
    'proband_GQ': int,
    'proband_DP': int,
    'father_DP': int,
    'mother_DP': int,
    'variant_MQ': float,
    'CADD_phred': float,
    'gnomad_genomes_312_AF': float
}

for i in vep_header:
    convert_dict[i] = str

for i in spliceAI_header:
    convert_dict[i] = str
    
for i in greenvaran_header:
    convert_dict[i] = str

df['gnomad_genomes_312_AF'] = df['gnomad_genomes_312_AF'].replace('.','0')
df = df[df['filter'] == 'PASS']
df = df[df['family'] != '8463'] # remove because of problems with this family
df = df[df['father_DP'] != '.']
df = df[df['mother_DP'] != '.']
df = df[df['proband_DP'] != 0]
df = df[df['mother_DP'] != 0]
df = df[df['father_DP'] != 0]
df = df.astype(convert_dict)
# df = df[df['gnomad_genomes_312_AF'] <= 0.001]
df = df[df['conf'] == 'hiConfDeNovo']

In [6]:
df['DS_AG'] = df['DS_AG'].replace('',0).replace('.',0)
df['DS_AL'] = df['DS_AL'].replace('',0).replace('.',0)
df['DS_DG'] = df['DS_DG'].replace('',0).replace('.',0)
df['DS_DL'] = df['DS_DL'].replace('',0).replace('.',0)
df['CADD_phred'] = df['CADD_phred'].replace('',0).replace('.',0)
df = df.astype({'DS_AG': float, 'DS_AL': float, 'DS_DG': float, 'DS_DL': float, 'CADD_phred': float})
df['SpliceAI_max'] = df[['DS_AG','DS_AL','DS_DG','DS_DL']].max(axis=1).astype(float)

In [7]:
# binomial test for chcecking the likelyness of heterozygous mutation having the Allele distribution is has
def binom_het_test(s):
    ref_count = int(s.split(':')[1].split(',')[0])
    alt_count = int(s.split(':')[1].split(',')[1])
    return binom_test(alt_count, alt_count + ref_count, p=0.5)

df['binom_p_val'] = df['proband_info'].apply(binom_het_test)

In [8]:
df_backup = df

In [9]:
df = df_backup

In [10]:
df.head()
print (len(df))

106889


In [11]:
#Filter
print ("Before filtering:", len(df))
df = df[df['binom_p_val'] > 0.05]
print ("after binom:", len(df))
df = df[df['proband_DP'] >= 12]
df = df[df['father_DP'] >= 12]
df = df[df['mother_DP'] >= 12]
print ("after DP:", len(df))
df = df[df['variant_MQ'] >= 30]
print ("after MQ:", len(df))
df = df[df['filter'] == 'PASS']
print ("after VQSR filter:", len(df))
 #GQ already applied with highconfDenovo

Before filtering: 106889
after binom: 63273
after DP: 25437
after MQ: 25427
after VQSR filter: 25427


In [12]:
#Filter
print ("Before filtering:", len(df))
df = df[df['binom_p_val'] > 0.01]
print ("after binom:", len(df))
df = df[df['proband_DP'] >= 12]
df = df[df['father_DP'] >= 12]
df = df[df['mother_DP'] >= 12]
print ("after DP:", len(df))
df = df[df['variant_MQ'] >= 30]
print ("after MQ:", len(df))
df = df[df['filter'] == 'PASS']
print ("after VQSR filter:", len(df))
 #GQ already applied with highconfDenovo

Before filtering: 25427
after binom: 25427
after DP: 25427
after MQ: 25427
after VQSR filter: 25427


In [14]:
len(df)
df.head()
df = df[df['gnomad_genomes_312_AF'] < 0.001]
print ("after gnomad filter:", len(df))

after gnomad filter: 19986


# DO postfilter

In [15]:
df.to_csv('Intermediate/For_postfilters.tsv',sep='\t',index=False, header= True)

## 1. Remove if DNM are not 0/0 in any other parents

In [16]:
#1-a extract locus for extraction from gvcf

f_final = open("Intermediate/For_postfilters.tsv", 'r')
out_locus = open("Intermediate/For_postfilters.locus", 'w')


for line in f_final:
    if 'locus' not in line:
        locus = line.split('\t')[0]
        CHROM = locus.split(':')[0]
        start = locus.split(':')[1]
        out_locus.write(CHROM + '\t' + start + '\t' + start + '\n')

In [17]:
#1-b extract GT of all parents from vcfs
"""
bcftools view -R Intermediate/For_postfilters.locus \
/projects/ps-gleesonlab8/User/arzoo/20230810_Jiny_Rerun_DNM_WGS_GMKF/1.Processing//output/merged_vcfs/annovar/joint.multisplit.VQSR.CGP.ann.vep.DNM.spliceAI.annotated.hg38_multianno.txt.reorder.vcf \
> Intermediate/For_postfilters.extracted.vcf"""

'\nbcftools view -R Intermediate/For_postfilters.locus /projects/ps-gleesonlab8/User/arzoo/20230810_Jiny_Rerun_DNM_WGS_GMKF/1.Processing//output/merged_vcfs/annovar/joint.multisplit.VQSR.CGP.ann.vep.DNM.spliceAI.annotated.hg38_multianno.txt.reorder.vcf > Intermediate/For_postfilters.extracted.vcf'

In [18]:
#2-c list up if a dnm candidate was shown from any mother or father 
f = open("Intermediate/For_postfilters.extracted.vcf", "r")

l_out_align = []


for line in f:
    if line[:2] == '##':
        pass
    elif line[:6] == '#CHROM':
        header = line.strip().split('\t')
       # print (header)
    else:
        d = dict(zip(header, line.strip().split('\t')))
        l = []
        for each in d:
            if '0/1' in d[each] or '1/1' in d[each]:
                if each != 'INFO' and int(d[each].split(':')[2]) >= 12: #If dp >= 12 
                    l.append(each)
        dic_pro_member = {}
        #print (l)
        if len(l) > 1: #if anyone else has same variant as DNM
            locus = line.split('\t')[0] + ':' + line.split('\t')[1]
            ped = open("/projects/ps-gleesonlab8/User/arzoo/GMKF_full_ped/complete_peds/20230216_GMKF_NTD_WGS_trios_fixed.ped", 'r' )
            for line in ped:
                s = line.split('\t')
                Family = s[0]
                proband = s[1]
                father = s[2]
                mother = s[3]
                #if father != '0': #if proband
                #    dic_pro_member[Family] = {'locus':locus, 'member':[]}
                if proband in l: #proband
                    dic_pro_member[Family] = {'locus':locus, 'member':[]}
                    dic_pro_member[Family]['member'].append('proband')
                if father in l: #father
                    dic_pro_member[Family] = {'locus':locus, 'member':[]}
                    dic_pro_member[Family]['member'].append('father')
                if mother in l: #mother
                    dic_pro_member[Family] = {'locus':locus, 'member':[]}
                    dic_pro_member[Family]['member'].append('mother')
        
        for family in dic_pro_member:
            members = dic_pro_member[family]['member']
            locus = dic_pro_member[family]['locus']
            if 'father' in members and 'proband' not in members:
                l_out_align.append(locus)
            elif 'mother' in members and 'proband' not in members:
                l_out_align.append(locus)
                
l_out_align = list(set(l_out_align))
print (len(l_out_align))

f_final = open("Intermediate/For_postfilters.tsv", 'r')


out_filter1 = open("Intermediate/For_postfilters.PF1.tsv", 'w')
cnt_all = 0
cnt = 0
for line in f_final:
    if '#' in line:
        out_filter1.write(line)
    else:
        cnt_all += 1
        locus = line.split('\t')[0]
        #print (locus)
        
        if locus not in l_out_align:
            cnt +=1
            out_filter1.write(line)

out_filter1.close()   
print ("Before PF1:", cnt_all)
print ("After PF1:", cnt)
print ("Removed form PF1:", cnt_all-cnt)

9083
Before PF1: 19987
After PF1: 15667
Removed form PF1: 4320


In [19]:
print (len(l_out_align))

9083


## 2. Remove if DNM are clustered within 10bp

In [20]:
#2. remove homopolymer

f_pf1 = open("Intermediate/For_postfilters.PF1.tsv", 'r')
out_pf2 = open("Intermediate/For_postfilters.PF2.tsv", 'w')

dic_pro_locus = {}

l_homo = []
for line in f_pf1:
    if 'locus' in line:
        pass
    else:
        s = line.split('\t')
        locus = s[0]
        pro = s[4]
        if pro not in dic_pro_locus:
            dic_pro_locus[pro] = [locus]
        else:
            dic_pro_locus[pro].append(locus)

for pro in dic_pro_locus:
    if len(dic_pro_locus[pro]) > 1:
        l = dic_pro_locus[pro]
        #print (l)
        base_chr = l[0].split(':')[0]
        base_pos = int(l[0].split(':')[1])
        for i in range(1,len(l)):
            if l[i].split(':')[0] != base_chr:
                base_chr = l[i].split(':')[0]
                base_pos = int(l[i].split(':')[1])
            else:
                if int(l[i].split(':')[1]) - int(base_pos) < 10 :
                    l_homo.append(base_chr + ':' + str(base_pos))
                    base_chr = l[i].split(':')[0]
                    base_pos = l[i].split(':')[1]
                    l_homo.append(base_chr + ':' + str(base_pos))
                else:
                    base_chr = l[i].split(':')[0]
                    base_pos = l[i].split(':')[1]
f_pf1.close()
l_homo = list(set(l_homo))
print (len(l_homo))
cnt = 0
cnt_all =0
#print (l_homo)
f_pf1 = open("Intermediate/For_postfilters.PF1.tsv", 'r')
for line in f_pf1:
    if 'locus' in line:
        out_pf2.write(line)
    else:
        cnt_all +=1
        if line.split('\t')[0] not in l_homo:
            out_pf2.write(line)
            cnt+=1
        else:
            pass
            
print ("Before PF2:", cnt_all)
print ("After PF2:", cnt)
print ("Removed from PF2:", cnt_all-cnt)
out_pf2.close()

1789
Before PF2: 15666
After PF2: 13853
Removed from PF2: 1813


In [21]:
df = pd.read_csv("Intermediate/For_postfilters.PF2.tsv", sep = '\t')
df.head()

/scratch/hiyoothere/34125475.tscc-mgr7.local/ipykernel_171978/1831293005.py:1: DtypeWarning: Columns (78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Intermediate/For_postfilters.PF2.tsv", sep = '\t')


,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val
0,chr1:911634,G,A,PASS,GLE_8438071884,6411,hiConfDeNovo,"0/1:20,13:33:99:83:23:332,0,506:247,0,594","0/0:39,0:39:67:83:23:0,102,1265:0,67,1257","0/0:30,0:30:49:83:23:0,84,1260:0,49,1252",99,33,39,30,60.00,A,upstream_gene_variant,MODIFIER,NaN,ENSG00000241180,Transcript,ENST00000398216,lncRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2537.0,1.0,NaN,SNV,NaN,NaN,YES,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,"44551_pro,117891_enh,752_biv","bivalent,enhancer,promoter","JungEtAl2019,SegWey,ENCODE-HMM","SAMD11,AL645608.6,AL645608.2",0.554292,2.0,0.0,0.000007,0.0,0.296206
1,chr1:1028565,TTCCGAAGGAACCGAGCCCCAGCCCCTCGTGG,*,PASS,GLE_6317135658,8088,hiConfDeNovo,"0/1:20,12:32:99:.:.:.:.:470,0,775:448,0,801","0/0:16,0:26:70:.:.:0|1:1028565_T_C:255,303,110...","0/0:45,0:45:23:.:.:.:.:0,1,1015:0,23,1062",99,32,26,45,54.41,-,intron_variant,MODIFIER,AGRN,ENSG00000188157,Transcript,ENST00000379370,protein_coding,NaN,2/35,ENST00000379370.7:c.463+6103_463+6163del,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,deletion,HGNC,HGNC:329,YES,NM_198576.4,NaN,1.0,P1,CCDS30551.1,ENSP00000368678,O00468.199,NaN,UPI00001D7C8B,O00468-6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,126243_enh,enhancer,FOCS,AGRN,0.026044,3.0,0.0,0.000000,0.0,0.215327
2,chr1:1062103,C,*,PASS,GLE_1627425919,8154,hiConfDeNovo,"0/1:16,14:31:99:55:2:.:.:540,0,616:461,0,702","0/0:24,0:24:25:55:2:.:.:0,57,855:0,25,850","0/0:37,0:37:30:55:2:.:.:0,60,900:0,30,897",99,31,24,37,60.00,G,intron_variant,MODIFIER,AGRN,ENSG00000188157,Transcript,ENST00000379370,protein_coding,NaN,31/35,ENST00000379370.7:c.5371-41C>G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:329,YES,NM_198576.4,NaN,1.0,P1,CCDS30551.1,ENSP00000368678,O00468.199,NaN,UPI00001D7C8B,O00468-6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,"128024_enh,47647_pro,47653_pro,128037_enh,1279...","enhancer,promoter","DECRES,FOCS,JungEtAl2019,BENGI,EnsemblRegBuild","RNF223,AL390719.1,AGRN",0.540841,2.0,0.0,0.000000,0.0,0.855536
3,chr1:1065285,G,A,PASS,GLE_6811993413,7671,hiConfDeNovo,"0/1:7,5:12:89:-1:-1:0|1:1065263_C_G:113,0,203:...","0/0:22,0:22:25:.:.:.:.:0,0,0:0,25,52","0/0:22,0:22:23:-1:-1:.:.:0,0,178:0,23,230",89,12,22,22,59.60,A,intron_

In [187]:
def sort_Consequences(s):
    return '&'.join(sorted(s.split('&')))

df['Consequence'] = df['Consequence'].apply(sort_Consequences)

In [23]:
#1-a extract locus for extraction from gvcf

f_final = open("Intermediate/For_postfilters.PF2.tsv", 'r')
out_locus = open("Intermediate/For_postfilters.PF2.locus", 'w')


for line in f_final:
    if 'locus' not in line:
        locus = line.split('\t')[0]
        CHROM = locus.split(':')[0]
        start = locus.split(':')[1]
        out_locus.write(CHROM + '\t' + start + '\t' + start + '\n')

In [24]:
#Segdup and clustered regions removed
f = open("Intermediate/For_postfilters.PF2.no_segdup_clustered.locus", 'r')
l = []
for line in f:
    s = line.split('\t')
    l.append(s[0] + ':' + s[1])
f.close()

In [33]:
f = open("Intermediate/For_postfilters.PF2.tsv", 'r')
out = open("Intermediate/For_postfilters.PF2.region_filtered.tsv", 'w')
for line in f:
    if 'locus' in line:
        out.write(line)
    else:
        s = line.split('\t')
        locus = s[0]
       # print (locus)
        if locus in l:
            out.write(line)

chr1:911634
chr1:1028565
chr1:1062103
chr1:1065285
chr1:1081038
chr1:1099730
chr1:1134747
chr1:1134761
chr1:1147323
chr1:1153310
chr1:1165686
chr1:1206547
chr1:1224252
chr1:1329543
chr1:1421148
chr1:1428223
chr1:1605436
chr1:1668650
chr1:1863087
chr1:1970069
chr1:1979062
chr1:1982114
chr1:2148288
chr1:2177554
chr1:2439934
chr1:2522886
chr1:2653990
chr1:2654010
chr1:2656365
chr1:2695822
chr1:2699571
chr1:2750253
chr1:2750284
chr1:2750284
chr1:2751158
chr1:2820345
chr1:2852534
chr1:2853616
chr1:2896383
chr1:2909887
chr1:3025173
chr1:3095327
chr1:3181666
chr1:3222216
chr1:3277757
chr1:3284972
chr1:3354458
chr1:3677070
chr1:3715199
chr1:3797725
chr1:3860504
chr1:3929230
chr1:3929300
chr1:4046140
chr1:4471312
chr1:4894275
chr1:5110244
chr1:5712178
chr1:5712188
chr1:5712231
chr1:5746768
chr1:5748556
chr1:5802831
chr1:5816837
chr1:5817172
chr1:5843917
chr1:5988104
chr1:5999986
chr1:6006752
chr1:6305324
chr1:6343244
chr1:6401157
chr1:6519895
chr1:7104234
chr1:7177460
chr1:7303191
chr1:7548899


chr5:7342103
chr5:7371566
chr5:7557192
chr5:7800741
chr5:7824317
chr5:7835395
chr5:7847780
chr5:7917024
chr5:8007042
chr5:8419114
chr5:8749415
chr5:8798069
chr5:8939239
chr5:9384563
chr5:9473208
chr5:9861035
chr5:10195891
chr5:10337299
chr5:10346080
chr5:10452313
chr5:10658255
chr5:10681690
chr5:10730548
chr5:10731119
chr5:10911520
chr5:11045988
chr5:11087896
chr5:11143841
chr5:11181818
chr5:11311797
chr5:11508552
chr5:11636727
chr5:12380748
chr5:12410624
chr5:12465397
chr5:12607759
chr5:12659849
chr5:13238824
chr5:13313608
chr5:14099891
chr5:14258576
chr5:14605437
chr5:14937711
chr5:15074851
chr5:15686834
chr5:15746598
chr5:15748061
chr5:16311800
chr5:16365842
chr5:16791955
chr5:16818402
chr5:17014910
chr5:17100860
chr5:17104408
chr5:17332374
chr5:17600317
chr5:17942947
chr5:18674118
chr5:18943689
chr5:18954105
chr5:19205787
chr5:19359006
chr5:20437522
chr5:20452974
chr5:20708968
chr5:20829228
chr5:20907432
chr5:21188250
chr5:21207625
chr5:21710339
chr5:22783797
chr5:23633385
chr5:237

chr13:109522805
chr13:110394113
chr13:110474664
chr13:110497940
chr13:110629704
chr13:110915619
chr13:110915619
chr13:111326419
chr13:111569477
chr13:111843451
chr13:111918706
chr13:112079795
chr13:112149187
chr13:112209807
chr13:112308409
chr13:112308882
chr13:112315553
chr13:112493019
chr13:112749966
chr13:112766772
chr13:112790382
chr13:112790398
chr13:112840198
chr13:112894293
chr13:112988760
chr13:113011676
chr13:113011687
chr13:113033000
chr13:113232173
chr13:113249214
chr13:113249227
chr13:113336887
chr13:113352314
chr13:113352360
chr13:113357138
chr13:113379185
chr13:113436161
chr13:113445897
chr13:113512084
chr13:113516565
chr13:113533233
chr13:113533272
chr13:113539559
chr13:113543114
chr13:113565292
chr13:113665544
chr13:113725259
chr13:113733999
chr13:113734014
chr13:113752437
chr13:113817476
chr13:113841676
chr13:113876762
chr13:113947195
chr13:114043864
chr13:114110990
chr13:114119213
chr13:114295028
chr14:16101952
chr14:16104134
chr14:18496176
chr14:19225374
chr14:193104

In [41]:
df = pd.read_csv("Intermediate/For_postfilters.PF2.region_filtered.tsv", sep = '\t')
df.head()

/scratch/hiyoothere/34125475.tscc-mgr7.local/ipykernel_171978/2984199825.py:1: DtypeWarning: Columns (78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Intermediate/For_postfilters.PF2.region_filtered.tsv", sep = '\t')


,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val
0,chr1:911634,G,A,PASS,GLE_8438071884,6411,hiConfDeNovo,"0/1:20,13:33:99:83:23:332,0,506:247,0,594","0/0:39,0:39:67:83:23:0,102,1265:0,67,1257","0/0:30,0:30:49:83:23:0,84,1260:0,49,1252",99,33,39,30,60.00,A,upstream_gene_variant,MODIFIER,NaN,ENSG00000241180,Transcript,ENST00000398216,lncRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2537.0,1.0,NaN,SNV,NaN,NaN,YES,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,"44551_pro,117891_enh,752_biv","bivalent,enhancer,promoter","JungEtAl2019,SegWey,ENCODE-HMM","SAMD11,AL645608.6,AL645608.2",0.554292,2.0,0.0,0.000007,0.0,0.296206
1,chr1:1028565,TTCCGAAGGAACCGAGCCCCAGCCCCTCGTGG,*,PASS,GLE_6317135658,8088,hiConfDeNovo,"0/1:20,12:32:99:.:.:.:.:470,0,775:448,0,801","0/0:16,0:26:70:.:.:0|1:1028565_T_C:255,303,110...","0/0:45,0:45:23:.:.:.:.:0,1,1015:0,23,1062",99,32,26,45,54.41,-,intron_variant,MODIFIER,AGRN,ENSG00000188157,Transcript,ENST00000379370,protein_coding,NaN,2/35,ENST00000379370.7:c.463+6103_463+6163del,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,deletion,HGNC,HGNC:329,YES,NM_198576.4,NaN,1.0,P1,CCDS30551.1,ENSP00000368678,O00468.199,NaN,UPI00001D7C8B,O00468-6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,126243_enh,enhancer,FOCS,AGRN,0.026044,3.0,0.0,0.000000,0.0,0.215327
2,chr1:1062103,C,*,PASS,GLE_1627425919,8154,hiConfDeNovo,"0/1:16,14:31:99:55:2:.:.:540,0,616:461,0,702","0/0:24,0:24:25:55:2:.:.:0,57,855:0,25,850","0/0:37,0:37:30:55:2:.:.:0,60,900:0,30,897",99,31,24,37,60.00,G,intron_variant,MODIFIER,AGRN,ENSG00000188157,Transcript,ENST00000379370,protein_coding,NaN,31/35,ENST00000379370.7:c.5371-41C>G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:329,YES,NM_198576.4,NaN,1.0,P1,CCDS30551.1,ENSP00000368678,O00468.199,NaN,UPI00001D7C8B,O00468-6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,"128024_enh,47647_pro,47653_pro,128037_enh,1279...","enhancer,promoter","DECRES,FOCS,JungEtAl2019,BENGI,EnsemblRegBuild","RNF223,AL390719.1,AGRN",0.540841,2.0,0.0,0.000000,0.0,0.855536
3,chr1:1065285,G,A,PASS,GLE_6811993413,7671,hiConfDeNovo,"0/1:7,5:12:89:-1:-1:0|1:1065263_C_G:113,0,203:...","0/0:22,0:22:25:.:.:.:.:0,0,0:0,25,52","0/0:22,0:22:23:-1:-1:.:.:0,0,178:0,23,230",89,12,22,22,59.60,A,intron_

In [42]:
total_size = 59281518
region_ratio = 3200000000/total_size

cutoff_wgs = 7* region_ratio
print (cutoff_wgs)

print (len(df))
df = df[df['proband'] != 'GLE_1100638258'] #proband that has over 900 DNMs
print (len(df))

377.8580703685759
11543
10793


In [43]:
df.to_csv('Intermediate/For_postfilters.PF2.region_filtered.countF.tsv',sep='\t',index=False, header= True)

In [ ]:
coding_term = c('frameshift_variant', 'inframe_deletion', 'inframe_insertion', 
                'synonymous_variant', 'missense_variant', 
                'splice_acceptor_variant', 'splice_donor_variant', 
                'start_lost', 
                'stop_gained', 'stop_lost', 'stop_retained_variant', "protein_altering_variant")

In [62]:
df_noncoding = df[(df['Consequence'] == 'downstream_gene_variant') | (df['Consequence'] == 'intergenic_variant')]

In [68]:
df_noncoding = df[(df['Consequence'] == 'downstream_gene_variant') |
                 (df['Consequence'] == 'intergenic_variant') |
                (df['Consequence'] == 'intron_variant') |
                 (df['Consequence'] == 'non_coding_transcript_exon_variant') | 
                 (df['Consequence'] == 'regulatory_region_variant') | 
                 (df['Consequence'] == 'upstream_gene_variant')  ]

print (len(df_noncoding))
df_noncoding.to_csv('Final_noncoding.tsv',sep='\t',index=False, header= True)
df_coding = df[(df['Consequence'] != 'downstream_gene_variant') &
                 (df['Consequence'] != 'intergenic_variant') &
                (df['Consequence'] != 'intron_variant') &
                 (df['Consequence'] != 'non_coding_transcript_exon_variant') &
                 (df['Consequence'] != 'regulatory_region_variant') &
                 (df['Consequence'] != 'upstream_gene_variant')  ]
print (len(df_coding))
df_coding.to_csv('Final_coding.tsv',sep='\t',index=False, header= True)

8817
1976


# IMPACT HIGH

In [44]:
impact_high = df[df['IMPACT'] == 'HIGH']

In [45]:
impact_high.head()

,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val
163,chr1:23077336,C,T,PASS,GLE_1567252581,8078,hiConfDeNovo,"0/1:23,11:34:99:81:21:280,0,697:195,0,785","0/0:37,0:37:65:81:21:0,100,1252:0,65,1244","0/0:32,0:32:47:81:21:0,82,1013:0,47,1005",99,34,37,32,60.00,T,stop_gained,HIGH,KDM1A,ENSG00000004487,Transcript,ENST00000356634,protein_coding,14/19,NaN,ENST00000356634.7:c.1771C>T,ENSP00000349049.3:p.Arg591Ter,1920,1771,591,R/*,Cga/Tga,COSV63088329,NaN,1.0,NaN,SNV,HGNC,HGNC:29079,NaN,NaN,NaN,1.0,NaN,CCDS30627.1,ENSP00000349049,O60341.213,NaN,UPI000020466D,O60341-1,1.0,NaN,NaN,PDB-ENSP_mappings:2dw4.A&PDB-ENSP_mappings:2ej...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,38.0,invalid_field,invalid_field,invalid_field,invalid_field,NaN,NaN,invalid_field,.&.&.&.,.&.&.&.,invalid_field,invalid_field,NaN,NaN,NaN,invalid_field,invalid_field,0.00,0.00,0.08,0.01,25,-5,46,24,72086_enh,enhancer,JungEtAl2019,"KDM1A,AL031428.1",0.721060,4.0,0.0,0.000000,0.08,0.057613
3063,chr4:15004125,C,CGGGGGGGGGGG,PASS,GLE_5959796249,8543,hiConfDeNovo,"0/1:10,5:15:48:68:9:0|1:15004125_C_CGGGGGGGGGG...","0/0:46,0:46:80:68:9:.:.:0,114,1710:0,80,1703","0/0:28,0:28:34:68:9:.:.:0,69,1035:0,34,1027",48,15,46,28,59.97,GGGGGGGGGGG,frameshift_variant,HIGH,CPEB2,ENSG00000137449,Transcript,ENST00000345451,protein_coding,1/10,NaN,ENST00000345451.8:c.1454_1455insGGGGGGGGGGG,ENSP00000334058.4:p.Phe488GlyfsTer22,1452-1453,1452-1453,484-485,-/GGGX,-/GGGGGGGGGGG,NaN,NaN,1.0,NaN,insertion,HGNC,HGNC:21745,NaN,NaN,NaN,1.0,A2,NaN,ENSP00000334058,NaN,A0A5K1VW61.9,UPI0001D04352,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,49,-21,-21,36,"574328_pro,354588_pro,975927_enh,975925_enh,97...","enhancer,promoter","DECRES,FOCS,SegWey,ENCODE-HMM,BENGI,EnsemblReg...","CPEB2-DT,CPEB2",0.285327,2.0,0.0,0.000007,0.00,0.301758
4560,chr5:160395471,C,T,PASS,GLE_5358917754,7651,hiConfDeNovo,"0/1:18,18:36:99:95:35:507,0,487:422,0,575","0/0:35,0:35:64:95:35:0,99,1485:0,64,1477","0/0:36,0:36:64:95:35:0,99,1321:0,64,1313",99,36,35,36,60.00,T,stop_gained,HIGH,ZBED8,ENSG00000221886,Transcript,ENST00000408953,protein_coding,2/2,NaN,ENST00000408953.4:c.20G>A,ENSP00000386184.3:p.Trp7Ter,485,20,7,W/*,tGg/tAg,NaN,NaN,-1.0,NaN,SNV,HGNC,HGNC:30804,YES,NM_022090.5,NaN,2.0,P1,CCDS34283.1,ENSP00000386184,Q8IZ13.127,NaN,UPI00000741A3,NaN,NaN,NaN,NaN,AFDB-ENSP_mappings:AF-Q8IZ13-F1.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,invalid_field,invalid_field,invalid_field,invalid_field,NaN,NaN,invalid_field,.&.,.&.,invalid_field,invalid_field,N

In [46]:
len(impact_high)

13

In [47]:
# impact_high.to_csv('impact_high_all_genes.tsv',sep='\t',index=False)

# metaSVM Deleterious missense mutations

In [48]:
MetaSVM_pred_D = df[df['MetaSVM_pred'] == 'D']

In [49]:
MetaSVM_pred_D.head()

,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val
266,chr1:47416556,G,T,PASS,GLE_2816585663,5651,hiConfDeNovo,"0/1:16,19:35:99:72:13:525,0,451:440,0,539","0/0:36,0:36:38:72:13:0,73,1088:0,38,1080","0/0:41,0:41:74:72:13:0,109,1316:0,74,1308",99,35,36,41,60.0,T,missense_variant,MODERATE,FOXE3,ENSG00000186790,Transcript,ENST00000335071,protein_coding,1/1,NaN,ENST00000335071.4:c.241G>T,ENSP00000334472.2:p.Ala81Ser,272,241,81,A/S,Gcc/Tcc,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:3808,YES,NM_012186.3,NaN,NaN,P1,CCDS550.1,ENSP00000334472,Q13461.175,A0A0A1EII5.48,UPI000012ADD3,NaN,1.0,tolerated(0.16),possibly_damaging(0.703),AFDB-ENSP_mappings:AF-Q13461-F1.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.1,invalid_field,invalid_field,invalid_field,invalid_field,D,0.87335,invalid_field,0.885,T,invalid_field,invalid_field,NaN,NaN,NaN,invalid_field,invalid_field,0.00,0.00,0.0,0.00,7,-36,-17,0,"562852_pro,38851_pro,660_biv","bivalent,promoter","DECRES,SegWey,ENCODE-HMM,BENGI,EnsemblRegBuild",FOXE3,0.777246,4.0,0.0,0.0000,0.00,0.735879
890,chr1:230779122,G,T,PASS,GLE_4487097024,8346,hiConfDeNovo,"0/1:12,16:28:99:89:29:452,0,268:367,0,356","0/0:36,0:36:65:89:29:0,100,1170:0,65,1162","0/0:33,0:33:55:89:29:0,90,1107:0,55,1099",99,28,36,33,60.0,T,missense_variant,MODERATE,CAPN9,ENSG00000135773,Transcript,ENST00000271971,protein_coding,9/20,NaN,ENST00000271971.7:c.1103G>T,ENSP00000271971.2:p.Arg368Leu,1212,1103,368,R/L,cGc/cTc,rs144137595&COSV99680503,NaN,1.0,NaN,SNV,HGNC,HGNC:1486,YES,NM_006615.3,NaN,1.0,P1,CCDS1586.1,ENSP00000271971,O14815.179,NaN,UPI000006E882,O14815-1,NaN,deleterious(0.02),probably_damaging(0.999),AFDB-ENSP_mappings:AF-O14815-F1.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000465,0.000196,0.000185,0.000231,0.0,0.000000,0.000046,0.000327,0.0,0.000000,0.000465,EA,NaN,0&1,0&1,NaN,NaN,NaN,NaN,NaN,NaN,32.0,invalid_field,invalid_field,invalid_field,invalid_field,D,0.93601,invalid_field,0.999&0.999&1.0,D&D&D,invalid_field,invalid_field,49.0,0.000196,250454.0,invalid_field,invalid_field,0.00,0.00,0.0,0.04,11,-37,-29,15,69856_enh,enhancer,BENGI,"CAPN9,AL512328.1",0.691448,1.0,0.0,0.0002,0.04,0.571588
1992,chr2:232791057,C,T,PASS,GLE_8240152843,8252,hiConfDeNovo,"0/1:26,19:45:99:86:26:486,0,702:401,0,790","0/0:35,0:35:64:86:26:0,99,1147:0,64,1139","0/0:33,0:33:52:86:26:0,87,1305:0,52,1297",99,45,35,33,60.0,T,missense_variant,MODERATE,GIGYF2,ENSG00000204120,Transcript,ENST00000373563,protein_coding,11/29,NaN,ENST00000373563.9:c.980C>T,ENSP00000362664.5:p.Pro327Leu,1151,980,327,P/L,cCt/cTt,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:11960,YES,NM_001103146.3,NaN,1.0,P4,CCDS33401.1,ENSP00000362664,Q6Y7W6.155,NaN,UPI00001BD8AE,Q6Y7W6-1,1.0,deleterious(0.04),possibly_damaging(0.725),AFDB-ENSP_mappings:AF-Q6Y7W6-F1.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [50]:
len(MetaSVM_pred_D)

17

# PolyPhen SIFT CADD

In [51]:
DmisMC = df[df['PolyPhen'].str.contains("probably_damaging") | df['SIFT'].str.contains("deleterious\(") ]
DmisMC = DmisMC[DmisMC['CADD_phred'] >= 20 ]
len(DmisMC)

39

# SpliceAI

In [52]:
splice_AI = df[df['DS_AG'] != ''].astype({'DS_AG': float, 'DS_AL': float, 'DS_DG': float, 'DS_DL': float})
splice_AI.head()

,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val
0,chr1:911634,G,A,PASS,GLE_8438071884,6411,hiConfDeNovo,"0/1:20,13:33:99:83:23:332,0,506:247,0,594","0/0:39,0:39:67:83:23:0,102,1265:0,67,1257","0/0:30,0:30:49:83:23:0,84,1260:0,49,1252",99,33,39,30,60.00,A,upstream_gene_variant,MODIFIER,NaN,ENSG00000241180,Transcript,ENST00000398216,lncRNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2537.0,1.0,NaN,SNV,NaN,NaN,YES,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,"44551_pro,117891_enh,752_biv","bivalent,enhancer,promoter","JungEtAl2019,SegWey,ENCODE-HMM","SAMD11,AL645608.6,AL645608.2",0.554292,2.0,0.0,0.000007,0.0,0.296206
1,chr1:1028565,TTCCGAAGGAACCGAGCCCCAGCCCCTCGTGG,*,PASS,GLE_6317135658,8088,hiConfDeNovo,"0/1:20,12:32:99:.:.:.:.:470,0,775:448,0,801","0/0:16,0:26:70:.:.:0|1:1028565_T_C:255,303,110...","0/0:45,0:45:23:.:.:.:.:0,1,1015:0,23,1062",99,32,26,45,54.41,-,intron_variant,MODIFIER,AGRN,ENSG00000188157,Transcript,ENST00000379370,protein_coding,NaN,2/35,ENST00000379370.7:c.463+6103_463+6163del,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,deletion,HGNC,HGNC:329,YES,NM_198576.4,NaN,1.0,P1,CCDS30551.1,ENSP00000368678,O00468.199,NaN,UPI00001D7C8B,O00468-6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,126243_enh,enhancer,FOCS,AGRN,0.026044,3.0,0.0,0.000000,0.0,0.215327
2,chr1:1062103,C,*,PASS,GLE_1627425919,8154,hiConfDeNovo,"0/1:16,14:31:99:55:2:.:.:540,0,616:461,0,702","0/0:24,0:24:25:55:2:.:.:0,57,855:0,25,850","0/0:37,0:37:30:55:2:.:.:0,60,900:0,30,897",99,31,24,37,60.00,G,intron_variant,MODIFIER,AGRN,ENSG00000188157,Transcript,ENST00000379370,protein_coding,NaN,31/35,ENST00000379370.7:c.5371-41C>G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:329,YES,NM_198576.4,NaN,1.0,P1,CCDS30551.1,ENSP00000368678,O00468.199,NaN,UPI00001D7C8B,O00468-6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,"128024_enh,47647_pro,47653_pro,128037_enh,1279...","enhancer,promoter","DECRES,FOCS,JungEtAl2019,BENGI,EnsemblRegBuild","RNF223,AL390719.1,AGRN",0.540841,2.0,0.0,0.000000,0.0,0.855536
3,chr1:1065285,G,A,PASS,GLE_6811993413,7671,hiConfDeNovo,"0/1:7,5:12:89:-1:-1:0|1:1065263_C_G:113,0,203:...","0/0:22,0:22:25:.:.:.:.:0,0,0:0,25,52","0/0:22,0:22:23:-1:-1:.:.:0,0,178:0,23,230",89,12,22,22,59.60,A,intron_

In [53]:
splice_AI['SpliceAI_max'] = splice_AI[['DS_AG','DS_AL','DS_DG','DS_DL']].max(axis=1).astype(float)
splice_AI_high = splice_AI[splice_AI['SpliceAI_max'] > 0.5].sort_values(by='SpliceAI_max',ascending=False)

In [54]:
len(splice_AI_high)

9

# GREEN-VARAN level 4 regulatory mutations

In [55]:
df['greendb_level']

0        2.0
1        3.0
2        2.0
3        2.0
4        2.0
        ... 
11538    2.0
11539    2.0
11540    NaN
11541    1.0
11542    1.0
Name: greendb_level, Length: 10793, dtype: float64

In [56]:
greenvaran_high = df[df['greendb_level'] == 4]
greenvaran_high = greenvaran_high[greenvaran_high['IMPACT'] == 'MODIFIER']

In [57]:
greenvaran_genes = []
for i in list(greenvaran_high.greendb_genes):
    for j in i.split(','):
        greenvaran_genes.append(j)
Counter(greenvaran_genes).most_common()

[('TBC1D2', 2),
 ('TP73', 1),
 ('AL136528.1', 1),
 ('EPB41', 1),
 ('TMEM200B', 1),
 ('HPCA', 1),
 ('FAF1', 1),
 ('FAF1-AS1', 1),
 ('DPYD', 1),
 ('BCL9', 1),
 ('LMNA', 1),
 ('FASLG', 1),
 ('MIR29B2CHG', 1),
 ('CD34', 1),
 ('CENPF', 1),
 ('WDPCP', 1),
 ('AC096664.2', 1),
 ('RNU6-187P', 1),
 ('HNRNPA3', 1),
 ('AC074286.1', 1),
 ('SPEG', 1),
 ('THRB', 1),
 ('FOXP1', 1),
 ('EPHB3', 1),
 ('TP63', 1),
 ('AC010280.1', 1),
 ('PIK3R1', 1),
 ('RNF217', 1),
 ('NKAIN2', 1),
 ('RNF217-AS1', 1),
 ('PRKAG2', 1),
 ('AP001207.1', 1),
 ('GRHL2', 1),
 ('EIF3H', 1),
 ('LINC00536', 1),
 ('TPM2', 1),
 ('GBA2', 1),
 ('AL360081.1', 1),
 ('GABBR2', 1),
 ('SEC61B', 1),
 ('RN7SKP225', 1),
 ('ZEB1-AS1', 1),
 ('ZEB1', 1),
 ('MIR7152', 1),
 ('CDH23', 1),
 ('WAPL', 1),
 ('AL844892.2', 1),
 ('GRID1', 1),
 ('TCTN3', 1),
 ('SHANK2', 1),
 ('IL10RA', 1),
 ('LINC02450', 1),
 ('PRICKLE1', 1),
 ('GRK1', 1),
 ('LINC00552', 1),
 ('NRL', 1),
 ('GPATCH2L', 1),
 ('AC016526.4', 1),
 ('SLC24A1', 1),
 ('AC004381.4', 1),
 ('ACSM3', 1

In [58]:
len(greenvaran_high)

41

In [248]:
greenvaran_high.head()

,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val
61,chr1:3677070,T,C,PASS,GLE_1567252581,8078,hiConfDeNovo,"0/1:21,19:40:99:99:39:500,0,511:415,0,599","0/0:40,0:40:72:99:39:0,107,1371:0,72,1363","0/0:35,0:35:65:99:39:0,100,1211:0,65,1203",99,40,40,35,60.0,C,intron_variant,MODIFIER,TP73,ENSG00000078900,Transcript,ENST00000346387,protein_coding,NaN,1/11,ENST00000346387.8:c.-33-5263T>C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:12003,NaN,NaN,NaN,5.0,NaN,CCDS55568.1,ENSP00000340740,O15350.232,NaN,UPI000002B05C,O15350-6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,29,7,25,-2,"35137_pro,1487719_enh,35114_pro,89917_enh","enhancer,promoter","DECRES,FOCS,ENCODE-HMM,BENGI,FulcoEtAl2019","TP73,AL136528.1",0.865050,4.0,0.0,0.000000,0.0,0.874629
253,chr1:29117534,A,G,PASS,GLE_8739880167,8074,hiConfDeNovo,"0/1:15,21:36:99:84:25:622,0,413:537,0,501","0/0:33,0:33:50:84:25:0,85,1058:0,50,1050","0/0:51,0:51:78:84:25:0,113,1708:0,78,1700",99,36,33,51,60.0,G,3_prime_UTR_variant,MODIFIER,EPB41,ENSG00000159023,Transcript,ENST00000343067,protein_coding,21/21,NaN,ENST00000343067.9:c.*722A>G,NaN,3524,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:3377,YES,NM_001376013.1,NaN,5.0,P2,CCDS53288.1,ENSP00000345259,P11171.232,NaN,UPI000014177D,P11171-1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,23,-46,-1,-3,85346_enh,enhancer,BENGI,"EPB41,TMEM200B",0.935115,4.0,0.0,0.000000,0.0,0.405032
274,chr1:32893663,C,G,PASS,GLE_5368458628,8158,hiConfDeNovo,"0/1:8,7:15:99:-1:-1:188,0,203:163,0,291","0/0:32,0:32:99:-1:-1:0,90,1032:0,113,1084","0/0:54,0:54:25:.:.:0,0,0:0,25,52",99,15,32,54,60.0,G,downstream_gene_variant,MODIFIER,TMEM54,ENSG00000121900,Transcript,ENST00000329151,protein_coding,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs748820769,932.0,-1.0,NaN,SNV,HGNC,HGNC:24143,NaN,NaN,NaN,1.0,NaN,CCDS85954.1,ENSP00000328630,Q969K7.135,NaN,UPI0000074654,Q969K7-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000035,0.000073,0.000149,0.0,0.0,0.0,0.00002,0.0,0.0,0.000149,gnomAD_AMR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-44,-22,-30,-34,"579548_pro,88370_enh","enhancer,promoter","FOCS,SegWey,ENCODE-HMM,BENGI,EnsemblRegBuild",HPCA,0.727079,4.0,0.0,0.000007,0.0,1.000000
361,chr1:50451021,C,A,PASS,GLE_4931115797,8077,hiConfDeNovo,"0/1:21,17:38:99:97:37:457,0,616:372,0,704","0/0:41,0:41:64:97:37:0,99,1305:0,64,1297","0/0:38,0:38:67:97:37:0,102,1287:0,67,1279",99,38,41,38,60.0,A,intron_varia

# Combined

In [69]:
combined = pd.concat([impact_high,MetaSVM_pred_D,DmisMC,splice_AI_high,greenvaran_high]).drop_duplicates()
combined.head()

,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val
163,chr1:23077336,C,T,PASS,GLE_1567252581,8078,hiConfDeNovo,"0/1:23,11:34:99:81:21:280,0,697:195,0,785","0/0:37,0:37:65:81:21:0,100,1252:0,65,1244","0/0:32,0:32:47:81:21:0,82,1013:0,47,1005",99,34,37,32,60.00,T,stop_gained,HIGH,KDM1A,ENSG00000004487,Transcript,ENST00000356634,protein_coding,14/19,NaN,ENST00000356634.7:c.1771C>T,ENSP00000349049.3:p.Arg591Ter,1920,1771,591,R/*,Cga/Tga,COSV63088329,NaN,1.0,NaN,SNV,HGNC,HGNC:29079,NaN,NaN,NaN,1.0,NaN,CCDS30627.1,ENSP00000349049,O60341.213,NaN,UPI000020466D,O60341-1,1.0,NaN,NaN,PDB-ENSP_mappings:2dw4.A&PDB-ENSP_mappings:2ej...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,38.0,invalid_field,invalid_field,invalid_field,invalid_field,NaN,NaN,invalid_field,.&.&.&.,.&.&.&.,invalid_field,invalid_field,NaN,NaN,NaN,invalid_field,invalid_field,0.00,0.00,0.08,0.01,25,-5,46,24,72086_enh,enhancer,JungEtAl2019,"KDM1A,AL031428.1",0.721060,4.0,0.0,0.000000,0.08,0.057613
3063,chr4:15004125,C,CGGGGGGGGGGG,PASS,GLE_5959796249,8543,hiConfDeNovo,"0/1:10,5:15:48:68:9:0|1:15004125_C_CGGGGGGGGGG...","0/0:46,0:46:80:68:9:.:.:0,114,1710:0,80,1703","0/0:28,0:28:34:68:9:.:.:0,69,1035:0,34,1027",48,15,46,28,59.97,GGGGGGGGGGG,frameshift_variant,HIGH,CPEB2,ENSG00000137449,Transcript,ENST00000345451,protein_coding,1/10,NaN,ENST00000345451.8:c.1454_1455insGGGGGGGGGGG,ENSP00000334058.4:p.Phe488GlyfsTer22,1452-1453,1452-1453,484-485,-/GGGX,-/GGGGGGGGGGG,NaN,NaN,1.0,NaN,insertion,HGNC,HGNC:21745,NaN,NaN,NaN,1.0,A2,NaN,ENSP00000334058,NaN,A0A5K1VW61.9,UPI0001D04352,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,49,-21,-21,36,"574328_pro,354588_pro,975927_enh,975925_enh,97...","enhancer,promoter","DECRES,FOCS,SegWey,ENCODE-HMM,BENGI,EnsemblReg...","CPEB2-DT,CPEB2",0.285327,2.0,0.0,0.000007,0.00,0.301758
4560,chr5:160395471,C,T,PASS,GLE_5358917754,7651,hiConfDeNovo,"0/1:18,18:36:99:95:35:507,0,487:422,0,575","0/0:35,0:35:64:95:35:0,99,1485:0,64,1477","0/0:36,0:36:64:95:35:0,99,1321:0,64,1313",99,36,35,36,60.00,T,stop_gained,HIGH,ZBED8,ENSG00000221886,Transcript,ENST00000408953,protein_coding,2/2,NaN,ENST00000408953.4:c.20G>A,ENSP00000386184.3:p.Trp7Ter,485,20,7,W/*,tGg/tAg,NaN,NaN,-1.0,NaN,SNV,HGNC,HGNC:30804,YES,NM_022090.5,NaN,2.0,P1,CCDS34283.1,ENSP00000386184,Q8IZ13.127,NaN,UPI00000741A3,NaN,NaN,NaN,NaN,AFDB-ENSP_mappings:AF-Q8IZ13-F1.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,invalid_field,invalid_field,invalid_field,invalid_field,NaN,NaN,invalid_field,.&.,.&.,invalid_field,invalid_field,N

In [80]:
combined[combined['SYMBOL'].isin(WES_gene_list)]

,locus,ref,alt,filter,proband,family,conf,proband_info,father_info,mother_info,proband_GQ,proband_DP,father_DP,mother_DP,variant_MQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,CADD_phred,DisGeNET,MPC,MTR,Mastermind,MetaSVM_pred,MetaSVM_rankscore,Phenotypes,Polyphen2_HDIV_score,SIFT_pred,SplicAI,SpliceRegion,gnomAD_exomes_AC,gnomAD_exomes_AF,gnomAD_exomes_AN,pLI,pLI_values,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,greendb_id,greendb_stdtype,greendb_dbsource,greendb_genes,greendb_constraint,greendb_level,greendb_more_support,gnomad_genomes_312_AF,SpliceAI_max,binom_p_val,locus_ref_alt_proband
163,chr1:23077336,C,T,PASS,GLE_1567252581,8078,hiConfDeNovo,"0/1:23,11:34:99:81:21:280,0,697:195,0,785","0/0:37,0:37:65:81:21:0,100,1252:0,65,1244","0/0:32,0:32:47:81:21:0,82,1013:0,47,1005",99,34,37,32,60.0,T,stop_gained,HIGH,KDM1A,ENSG00000004487,Transcript,ENST00000356634,protein_coding,14/19,NaN,ENST00000356634.7:c.1771C>T,ENSP00000349049.3:p.Arg591Ter,1920,1771,591,R/*,Cga/Tga,COSV63088329,NaN,1.0,NaN,SNV,HGNC,HGNC:29079,NaN,NaN,NaN,1.0,NaN,CCDS30627.1,ENSP00000349049,O60341.213,NaN,UPI000020466D,O60341-1,1.0,NaN,NaN,PDB-ENSP_mappings:2dw4.A&PDB-ENSP_mappings:2ej...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,38.0,invalid_field,invalid_field,invalid_field,invalid_field,NaN,NaN,invalid_field,.&.&.&.,.&.&.&.,invalid_field,invalid_field,NaN,NaN,NaN,invalid_field,invalid_field,0.0,0.0,0.08,0.01,25,-5,46,24,72086_enh,enhancer,JungEtAl2019,"KDM1A,AL031428.1",0.721060,4.0,0.0,0.000000,0.08,0.057613,chr1:23077336_C_T_GLE_1567252581
4819,chr6:33670584,G,A,PASS,GLE_7507632649,8336,hiConfDeNovo,"0/1:16,8:24:99:77:18:196,0,454:111,0,542","0/0:29,0:29:43:77:18:0,78,1170:0,43,1162","0/0:40,0:40:79:77:18:0,114,1710:0,79,1702",99,24,29,40,60.0,A,splice_region_variant&intron_variant,LOW,ITPR3,ENSG00000096433,Transcript,ENST00000374316,protein_coding,NaN,20/58,ENST00000374316.9:c.2441+8G>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,SNV,HGNC,HGNC:6182,NaN,NaN,NaN,5.0,P1,CCDS4783.1,ENSP00000363435,Q14573.204,NaN,UPI000013CB74,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.90,0.00,1,26,1,-11,1174714_enh,enhancer,BENGI,"MIR3934,ITPR3",0.913404,2.0,0.0,0.000000,0.90,0.151590,chr6:33670584_G_A_GLE_7507632649
6692,chr8:101573569,T,C,PASS,GLE_8797717620,6673,hiConfDeNovo,"0/1:19,26:45:99:96:36:694,0,441:609,0,529","0/0:38,0:38:65:96:36:0,100,1335:0,65,1327","0/0:33,0:33:64:96:36:0,99,1107:0,64,1099",99,45,38,33,60.0,C,intron_variant,MODIFIER,GRHL2,ENSG00000083307,Transcript,ENST00000395927,protein_coding,NaN,5/15,ENST00000395927.1:c.687-99T>C,NaN,NaN,NaN,NaN,NaN,NaN,rs1364292836,NaN,1.0,NaN,SNV,HGNC,HGNC:2799,NaN,NaN,NaN,2.0,NaN,CCDS83312.1,ENSP00000379260,Q6ISB3.146,NaN,UPI000035CC51,Q6ISB3-2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,0.00,-6,47,2,-8,"1728007_enh,1289801_enh",enhancer,"DECRES,JungEtAl2019,BENGI","AP001207.1,GRHL2",0.856029,4.0,0.0,0.000007,0.00,0.371298,chr8:101573569_T_C_GLE_8797717620


In [71]:
combined_short = combined[['locus','ref','alt','proband','family','Consequence','IMPACT','SYMBOL','PolyPhen','MetaSVM_pred','SpliceAI_max']].drop_duplicates()

In [77]:
combined['locus_ref_alt_proband'] = combined['locus'] + '_' + combined['ref'] + '_' + combined['alt'] + '_' + combined['proband']
combined.to_csv('Final_HIGH_impact.tsv',sep='\t',index=False)

In [75]:
len(combined)

98

In [79]:
f_WES = open("/projects/ps-gleesonlab8/User/hiyoothere/NTD/5.Analysis/DNM/Burden/NTD_DamMC2_woINVALID.genes", 'r')
WES_gene_list = []
for line in f_WES:
    WES_gene_list.append(line.strip())